In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
np.NaN = np.nan
import pandas_ta as ta


In [2]:
df = pd.read_csv('/home/sacsresta/Documents/RESEARCH/Project/sentiment_categorical/merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')

In [4]:
df.drop(columns=['Ticker','headline'], inplace = True)

In [5]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class
0,2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive
1,2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative
2,2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral
3,2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative
4,2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral
...,...,...,...,...,...,...,...,...,...
2533,2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive
2534,2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative
2535,2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative
2536,2025-02-27,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,negative


In [10]:
def SSL(data, period):
    data['smaHigh'] = data['High'].rolling(window=period).mean()
    data['smaLow'] = data['Low'].rolling(window=period).mean()
    data['sslDown'] = data.apply(lambda row: row['smaHigh'] if row['Close'] < row['smaLow'] else row['smaLow'], axis=1)
    data['sslUp'] = data.apply(lambda row: row['smaLow'] if row['Close'] < row['smaLow'] else row['smaHigh'], axis=1)
    data['Trend'] = data.apply(lambda row: "buy" if row['smaLow'] == row['sslDown'] else "sell", axis=1)
    data['ssl_signal'] = (data['Trend'] != data['Trend'].shift(1)).astype(int)
    return data

In [30]:
ssl_df = SSL(data=df.copy(), period=10)
ssl_df.dropna(inplace=True)

In [31]:
ssl_df.reset_index(drop=True, inplace=True)

In [32]:
ssl_df

,Date,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,smaHigh,smaLow,sslDown,sslUp,Trend,ssl_signal
0,2015-01-15,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,negative,27.677500,26.896750,27.677500,26.896750,sell,0
1,2015-01-16,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,positive,27.581000,26.843000,27.581000,26.843000,sell,0
2,2015-01-20,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,neutral,27.589000,26.870250,26.870250,27.589000,buy,1
3,2015-01-21,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,negative,27.679750,26.961250,26.961250,27.679750,buy,0
4,2015-01-22,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,neutral,27.786500,27.036750,27.036750,27.786500,buy,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,242.618999,238.150000,238.150000,242.618999,buy,0
2525,2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,244.559999,239.921001,239.921001,244.559999,buy,0
2526,2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative,245.534999,241.021001,245.534999,241.021001,sell,1
2527,2025-02-27,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,negative,246.084999,241.659001,246.084999,241.659001,sell,0


In [33]:
def Signal_rule_sentiment(row):
    if row['ssl_signal'] == 1 and row['sentiment_class'] == 'positive' and row['Trend'] == 'buy':
        return 1
    elif row['ssl_signal'] == 1 and row['sentiment_class'] == 'negative' and row['Trend'] == 'sell':
        return -1
    else:
        return 0

# Apply the function row-wise
ssl_df['combined_signal'] = ssl_df.apply(Signal_rule_sentiment, axis=1)

In [35]:
ssl_df['combined_signal'].value_counts()

combined_signal
 0    2394
-1      89
 1      46
Name: count, dtype: int64

In [53]:
ssl_df.set_index('Date', inplace=True)

In [54]:
def get_ssl_signal():
    return ssl_df['ssl_signal'].values

def get_trend():
    return ssl_df['Trend'].values

def get_news_class():
    return ssl_df['sentiment_class'].values

def get_combined_signal():
    return ssl_df.combined_signal.values

In [55]:
get_news_class()

array(['negative', 'positive', 'neutral', ..., 'negative', 'negative',
       'neutral'], dtype=object)

In [ ]:
from backtesting import Backtest, Strategy


class CustomStrategy(Strategy):
    def init(self):

        self.signal = self.I(get_combined_signal)



    def next(self):
        if self.signal == 1:
            if not self.position:
                self.buy()

        elif self.signal == -1:
            if self.position:
                self.sell()


In [ ]:
bt = Backtest(ssl_df, CustomStrategy, cash=100000, commission=0.02)

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:1201: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/tmp/ipykernel_240430/3189608658.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(ssl_df, CustomStrategy, cash=10000, commission=0.02)


In [58]:
stats = bt.run()

Backtest.run:   0%|          | 0/2528 [00:00<?, ?it/s]

/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:955: UserWarning: time=73: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:955: UserWarning: time=76: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:955: UserWarning: time=95: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:955: UserWarning: time=97: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/home/sacsresta/miniconda3/envs/research/lib/python3.10/site-packages/backtesting/backtesting.py:955: UserWarning: time=130: Broker canceled the relative-sized 

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
stats

Start                                     0.0
End                                    2528.0
Duration                               2528.0
Exposure Time [%]                         0.0
Equity Final [$]                   75212.6717
Equity Peak [$]                   80555.64942
Return [%]                          652.12672
Buy & Hold Return [%]               805.59819
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                          -140.05834
Beta                                  0.98335
Max. Drawdown [%]                   -38.72877
Avg. Drawdown [%]                    -4.80805
Max. Drawdown Duration                  452.0
Avg. Drawdown Duration                26.5618
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                    